# 🚀 OpenCode LLM Server v6.2

**Based on [ollama-x-opencode](https://github.com/p-lemonish/ollama-x-opencode)**

**Key Fix:** Create a SAVED model with 16K context built-in!

**Recommended:** Use Qwen3 for best tool support

In [ ]:
#@title 1️⃣ Choose Model
# Qwen3 has best tool support according to community feedback
BASE_MODEL = "qwen3:8b"  #@param ["qwen3:8b", "qwen3:4b", "llama3.1:8b", "mistral:7b"]
CONTEXT_SIZE = 16384  #@param [8192, 16384, 32768]

# The model name we'll create with larger context
MODEL = f"{BASE_MODEL.split(':')[0]}:{BASE_MODEL.split(':')[1]}-{CONTEXT_SIZE//1024}k"
print(f"📦 Base: {BASE_MODEL}")
print(f"📦 Will create: {MODEL} (with {CONTEXT_SIZE} context)")

In [ ]:
#@title 2️⃣ Setup
!nvidia-smi --query-gpu=name,memory.free --format=csv 2>/dev/null || echo 'No GPU'
!curl -fsSL https://ollama.com/install.sh | sh 2>&1 | tail -1
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb 2>/dev/null
!pip install -q flask requests pexpect
print('\n✅ Ready')

In [ ]:
#@title 3️⃣ Start Ollama & Create Model with 16K Context
import subprocess, time, os
import pexpect

os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
os.environ['OLLAMA_KEEP_ALIVE'] = '-1'

print('Starting Ollama...')
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)

# Pull base model
print(f'\nDownloading {BASE_MODEL}...')
!ollama pull {BASE_MODEL}

# Create model with larger context using Modelfile
print(f'\n📝 Creating {MODEL} with {CONTEXT_SIZE} context...')

modelfile_content = f'''FROM {BASE_MODEL}
PARAMETER num_ctx {CONTEXT_SIZE}
'''

with open('/tmp/Modelfile', 'w') as f:
    f.write(modelfile_content)

!ollama create {MODEL} -f /tmp/Modelfile

print(f'\n✅ {MODEL} ready with {CONTEXT_SIZE} context!')

In [ ]:
#@title 4️⃣ API Server v6.2 - Direct Passthrough
from flask import Flask, request, jsonify, Response
import requests as req
import json, time, uuid

app = Flask(__name__)

# ══════════════════════════════════════════════════════════════════════════════
# DIRECT PASSTHROUGH - Let Ollama handle everything natively
# The model now has 16K context built-in, so tools should work!
# ══════════════════════════════════════════════════════════════════════════════

@app.route('/v1/models', methods=['GET'])
def list_models():
    return jsonify({'object': 'list', 'data': [{'id': MODEL, 'object': 'model', 'owned_by': 'ollama'}]})

@app.route('/v1/chat/completions', methods=['POST'])
def chat_completions():
    data = request.json or {}
    messages = data.get('messages', [])
    tools = data.get('tools', [])
    
    # Get user message for logging
    user_msg = ''
    for m in reversed(messages or []):
        if m.get('role') == 'user':
            content = m.get('content', '')
            if isinstance(content, list):
                for p in content:
                    if isinstance(p, dict) and p.get('type') == 'text':
                        user_msg = p.get('text', '')[:40]
                        break
            else:
                user_msg = str(content)[:40] if content else ''
            break
    
    tool_names = [t.get('function', {}).get('name', '') for t in tools]
    print(f"[{time.strftime('%H:%M:%S')}] '{user_msg}' | tools={tool_names[:4]}")
    
    # Build Ollama request - pass tools directly!
    ollama_payload = {
        'model': MODEL,
        'messages': messages,
        'stream': False,
    }
    
    # Pass tools to Ollama exactly as received
    if tools:
        ollama_payload['tools'] = tools
    
    try:
        result = req.post('http://localhost:11434/api/chat', json=ollama_payload, timeout=180).json()
    except Exception as e:
        print(f"  ❌ Ollama error: {e}")
        return jsonify({
            'id': f'chatcmpl-{uuid.uuid4().hex[:12]}',
            'object': 'chat.completion',
            'created': int(time.time()),
            'model': MODEL,
            'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Error, please try again.'}, 'finish_reason': 'stop'}]
        })
    
    message = result.get('message', {})
    content = message.get('content', '')
    tool_calls = message.get('tool_calls', [])
    
    # Build OpenAI-compatible response
    response_message = {'role': 'assistant'}
    
    if tool_calls:
        # Model returned tool calls natively!
        formatted_calls = []
        for i, tc in enumerate(tool_calls):
            func = tc.get('function', {})
            formatted_calls.append({
                'id': f'call_{uuid.uuid4().hex[:8]}',
                'type': 'function',
                'function': {
                    'name': func.get('name', ''),
                    'arguments': json.dumps(func.get('arguments', {})) if isinstance(func.get('arguments'), dict) else str(func.get('arguments', '{}'))
                }
            })
        response_message['content'] = None
        response_message['tool_calls'] = formatted_calls
        finish = 'tool_calls'
        print(f"  → tool: {formatted_calls[0]['function']['name']}")
    else:
        # Text response
        response_message['content'] = content or "I'll help you with that!"
        finish = 'stop'
        print(f"  → text: {len(content)} chars")
    
    return jsonify({
        'id': f'chatcmpl-{uuid.uuid4().hex[:12]}',
        'object': 'chat.completion',
        'created': int(time.time()),
        'model': MODEL,
        'system_fingerprint': 'fp_v62',
        'choices': [{'index': 0, 'message': response_message, 'logprobs': None, 'finish_reason': finish}],
        'usage': {'prompt_tokens': 100, 'completion_tokens': 50, 'total_tokens': 150}
    })

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'ok', 'version': '6.2', 'model': MODEL})

@app.route('/', methods=['GET'])
def root():
    return jsonify({'name': 'OpenCode LLM Server', 'version': '6.2', 'model': MODEL})

import threading
threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False), daemon=True).start()
time.sleep(2)
print('\n' + '='*55)
print(f'✅ Server v6.2 Ready')
print(f'   Model: {MODEL}')
print(f'   Context: {CONTEXT_SIZE} (built into model!)')
print('='*55)

In [ ]:
#@title 5️⃣ Test Tool Calling
import requests

# Test with OpenCode-like tools
test_tools = [
    {'type': 'function', 'function': {'name': 'write', 'description': 'Write content to a file', 'parameters': {'type': 'object', 'properties': {'filePath': {'type': 'string'}, 'content': {'type': 'string'}}, 'required': ['filePath', 'content']}}},
    {'type': 'function', 'function': {'name': 'bash', 'description': 'Run a shell command', 'parameters': {'type': 'object', 'properties': {'command': {'type': 'string'}}, 'required': ['command']}}},
    {'type': 'function', 'function': {'name': 'read', 'description': 'Read a file', 'parameters': {'type': 'object', 'properties': {'filePath': {'type': 'string'}}, 'required': ['filePath']}}},
]

print(f'Testing {MODEL}...\n')

r = requests.post('http://localhost:5000/v1/chat/completions', json={
    'model': MODEL,
    'messages': [{'role': 'user', 'content': 'Create a file called hello.py with print("hello world") in it'}],
    'tools': test_tools
}, timeout=180)

result = r.json()
msg = result['choices'][0]['message']

if msg.get('tool_calls'):
    tc = msg['tool_calls'][0]['function']
    print(f'✅ SUCCESS! Tool: {tc["name"]}')
    print(f'   Arguments: {tc["arguments"]}')
else:
    print(f'❌ Returned text instead of tool:')
    print(f'   {msg.get("content", "")[:200]}')

In [ ]:
#@title 6️⃣ Start Cloudflare Tunnel
import subprocess, re
from IPython.display import display, HTML

print('🌐 Starting Cloudflare Tunnel...\n')

tunnel = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

for line in tunnel.stdout:
    print(line, end='')
    if 'trycloudflare.com' in line:
        match = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if match:
            url = match.group()
            display(HTML(f'''
            <div style="background:linear-gradient(135deg,#7c3aed,#a855f7);padding:25px;border-radius:16px;margin:20px 0;color:white;font-family:system-ui">
                <h2 style="margin:0 0 10px 0">🚀 v6.2 Ready!</h2>
                <code style="background:rgba(0,0,0,0.2);padding:12px 16px;border-radius:8px;display:block;font-size:16px">{url}/v1</code>
                <p style="margin:15px 0 0 0;opacity:0.9">Model: {MODEL} | Context: {CONTEXT_SIZE} (built-in!)</p>
            </div>
            <div style="background:#fef3c7;color:#92400e;padding:12px 16px;border-radius:10px;font-size:14px">⚠️ Keep this tab open! Update opencode.json with model: "{MODEL}"</div>
            '''))
            break

print('\n📡 Tunnel logs:')
for line in tunnel.stdout:
    print(line, end='')